# ***MODOAP - Détection et extraction d'illustrations dans les pages de journaux / manuels / magazines***


Ce script permet de détecter et extraire les illustrations dans les pages d'un corpus spécifié. 
Il requiert :
  - des poids issus de l'entraînement préalable d'un modèle sur un corpus particulier
  - un corpus de journaux / manuels sur lequel opérer la détection : il s'agit d'un dossier contenant directement les pages des documents au format image.

Le script implémente la configuration décrite sur https://github.com/matterport/Mask_RCNN

**Ce script doit être lancé dans un environnement d'exécution GPU : Exécution -> Modifier le type d'exécution -> GPU**

In [ ]:
#@title ##  Préparation

#@markdown ### Synchronisation d'un Google Drive et installation de l'algorithme
#@markdown ##### Lancer cette cellule et cliquer sur le lien généré pour autoriser un compte Google Drive


import os
from google.colab import drive

if not os.path.exists("/content/drive/My Drive"):
  drive.mount('/content/drive')
else : print("Le Drive est déjà monté")


%cd
if not os.path.exists("/root/Mask_RCNN"):

  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  %cd /root/Mask_RCNN
  !pip install -q PyDrive
  !pip install -r requirements.txt
  !python setup.py install
  !cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py
  !sed -i -- 's/balloon/illustration/g' illustration.py
  !sed -i -- 's/Balloon/Illustration/g' illustration.py
 
else : print("L'algorithme est déjà téléchargé")

%tensorflow_version 1.x
!pip install q keras==2.1.5
!pip install q keras==2.1.5

%cd /root/Mask_RCNN/

import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob

ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

sys.path.append(ROOT_DIR) 
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration


In [ ]:
#@markdown ## Lancement de la détection d'illustrations sur le corpus
#@markdown #### Crée un fichier annotations_illustration.json dans le dossier corpus où sont stockées les régions des illustrations
#@markdown ---
#@markdown ### Entrer le chemin du répertoire où se trouve le corpus sur lequel opérer la détection :
dossier_corpus = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/Corpus/

#@markdown ---

#@markdown ### Entrer le chemin du fichier des poids (*.h5) issus de l'entraînement du modèle :
fichierpoids = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/MyDrive/Outils_Modoap/Detection_Illustrations/Poids/illustration20210303T1457/mask_rcnn_illustration_0200.h5

#@markdown ---
#@markdown ### Activer la visualisation des résultats (le temps de traitement est plus long, permet de vérifier les détections opérées) :
visualisation = "non" #@param ["oui", "non"]

IMAGE_DIR = dossier_corpus
config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()

warnings.filterwarnings("ignore")
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)
model.load_weights(fichierpoids, by_name=True)

corpus = [image for image in glob.glob(dossier_corpus+"/*.*") if not image.split(".")[-1].endswith("json")]
class_names = ['BG', 'illustration']
import cv2
import json
dicocorpus = {}
print("Lancement de la détection d'illustrations sur {} pages".format(len(corpus)))

compteur = 1
for cheminfichier in corpus :
  print("Page {} sur {}".format(compteur, len(corpus)))
  nomfichier = str(cheminfichier.split("/")[-1])
  image = cv2.imread(cheminfichier)
  results = model.detect([image], verbose=0)
  r = results[0]
  listeobjets = [vecteur for vecteur in r["rois"]]
  dicoimage={}
  compteur += 1

  if visualisation =="oui" :
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'])

  for i in range(len(r["rois"])) :
    vecteur_region = [str(coord) for coord in listeobjets[i]]
    classe = class_names[r["class_ids"][i]]
    score = str(r["scores"][i])
    dicoobjet = {"regions":vecteur_region, "classe" : classe, "score":score}
    dicoimage["objet"+str(i)] = dicoobjet
  dicocorpus[nomfichier] = dicoimage


with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "w") as dic :
  json.dump(dicocorpus, dic)

print("Les régions des illustrations détectées ont été enregitrées dans le fichier {}".format(os.path.join(dossier_corpus, "annotations_illustration.json")))


In [ ]:
import json

#@markdown ## Lancement de l'extraction des illustrations en fichiers .jpg
#@markdown #### Sauvegarde les illustrations détectées en fichiers .jpg à partir des informations stockées dans le fichier annotations_illustration.json  
#@markdown ---

#@markdown ### Spécifier un chemin de destination :
#@markdown #### Ou laisser vide pour extraire les illustrations dans un dossier nommé "illustrations_extraites" et situé dans le dossier du corpus
chemin_destination = "" #@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/magazines/extractions/

if not chemin_destination :
   chemin_destination = os.path.join(dossier_corpus,"illustrations_extraites")

if not os.path.exists(chemin_destination):
  os.mkdir(chemin_destination)

with open(os.path.join(dossier_corpus, "annotations_illustration.json"), "r") as mondic :
  dicocorpus2 = json.load(mondic)

print("Lancement des extractions")
for nomfichier in dicocorpus2 :
  cheminfichier = os.path.join(dossier_corpus, nomfichier)
  compteur = 0

  image = cv2.imread(cheminfichier)
  for objet in dicocorpus2[nomfichier].values():
    y1 = int(objet["regions"][0])
    y2 = int(objet["regions"][2])
    x1 = int(objet["regions"][1])
    x2 = int(objet["regions"][3])
    imagecropped = image[y1:y2, x1:x2] # [Y1:Y2, X1:X2] Le vecteur est sous forme [Y1 X1 Y2 X2]
    #cv2.imwrite("/content/drive/My Drive/Outils_Modoap/Detection_Illustrations/Illustrations_extraites/"+nomfichier+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg", imagecropped)
    cv2.imwrite(os.path.join(chemin_destination,nomfichier+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg"), imagecropped)
    chemin_sortie = os.path.join(chemin_destination,"".join(nomfichier.split(".")[:-1])+"_objet"+str(compteur)+"_"+objet["classe"]+".jpg") 
    cv2.imwrite(chemin_sortie, imagecropped)
    compteur += 1

print("Les illustrations ont été extraites dans le dossier {}".format(chemin_destination))


